# Data Preprocessing
This notebook handles data cleaning, merging datasets, encoding categorical variables, and preparing data for model training.

In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## Load Data

In [79]:
fred = pd.read_csv("../data/raw/fredgraph.csv")
rpp = pd.read_excel("../data/raw/rpp1224.xlsx", skiprows=5)
shed = pd.read_csv("../data/raw/public2024.csv")

rpp.columns = ['state', 'real_pce_2022', 'real_pce_2023', 'pce_pct_change',
               'real_income_2022', 'real_income_2023', 'income_pct_change']
rpp['state'] = rpp['state'].str.lower().str.strip()
rpp = rpp[~rpp['state'].isin(['united states', 'nan', ''])]
rpp = rpp.dropna(subset=['state'])

print(f"SHED shape: {shed.shape}")
print(f"RPP shape: {rpp.shape}")
print(f"FRED shape: {fred.shape}")

SHED shape: (12295, 753)
RPP shape: (55, 7)
FRED shape: (58, 13)


## Define Target Variable

Creating binary target: Can cover $400 emergency with cash/savings (1) vs Cannot cover (0)

We use EF3_c which asks if respondents can cover a $400 emergency 'with the money currently in my checking/savings account or with cash.' This represents true financial security - having liquid assets available for emergencies.

In [80]:
if 'EF3_c' in shed.columns:
    shed['can_cover_400'] = (shed['EF3_c'] == 'Yes').astype(int)
    
    print(f"\nTarget variable distribution:")
    print(shed['can_cover_400'].value_counts())
    print(f"\nPercentage who CAN cover: {shed['can_cover_400'].mean()*100:.1f}%")
    print(f"Percentage who CANNOT cover: {(1-shed['can_cover_400'].mean())*100:.1f}%")
else:
    print(f"Error: EF3_c column not found in dataset")
    print(f"Available EF3 columns: {[col for col in shed.columns if col.startswith('EF3')]}")


Target variable distribution:
can_cover_400
0    6927
1    5368
Name: count, dtype: int64

Percentage who CAN cover: 43.7%
Percentage who CANNOT cover: 56.3%


## Merge Datasets

Adding state-level economic indicators to individual SHED data

In [81]:
state_abbrev_to_name = {
    'al': 'alabama', 'ak': 'alaska', 'az': 'arizona', 'ar': 'arkansas',
    'ca': 'california', 'co': 'colorado', 'ct': 'connecticut', 'de': 'delaware',
    'fl': 'florida', 'ga': 'georgia', 'hi': 'hawaii', 'id': 'idaho',
    'il': 'illinois', 'in': 'indiana', 'ia': 'iowa', 'ks': 'kansas',
    'ky': 'kentucky', 'la': 'louisiana', 'me': 'maine', 'md': 'maryland',
    'ma': 'massachusetts', 'mi': 'michigan', 'mn': 'minnesota', 'ms': 'mississippi',
    'mo': 'missouri', 'mt': 'montana', 'ne': 'nebraska', 'nv': 'nevada',
    'nh': 'new hampshire', 'nj': 'new jersey', 'nm': 'new mexico', 'ny': 'new york',
    'nc': 'north carolina', 'nd': 'north dakota', 'oh': 'ohio', 'ok': 'oklahoma',
    'or': 'oregon', 'pa': 'pennsylvania', 'ri': 'rhode island', 'sc': 'south carolina',
    'sd': 'south dakota', 'tn': 'tennessee', 'tx': 'texas', 'ut': 'utah',
    'vt': 'vermont', 'va': 'virginia', 'wa': 'washington', 'wv': 'west virginia',
    'wi': 'wisconsin', 'wy': 'wyoming', 'dc': 'district of columbia'
}

shed['state_name'] = shed['ppstaten'].map(state_abbrev_to_name)

print(f"States successfully mapped: {shed['state_name'].notna().sum()}")
print(f"States not mapped: {shed['state_name'].isna().sum()}")

df = shed.merge(rpp, left_on='state_name', right_on='state', how='left')

print(f"\nMerged dataset shape: {df.shape}")
print(f"Records with RPP data: {df['pce_pct_change'].notna().sum()}")
print(f"Records without RPP data: {df['pce_pct_change'].isna().sum()}")

print(f"\nSample merged data:")
print(df[['ppstaten', 'state_name', 'pce_pct_change', 'real_income_2023']].head(10))

States successfully mapped: 12295
States not mapped: 0

Merged dataset shape: (12295, 762)
Records with RPP data: 12094
Records without RPP data: 201

Sample merged data:
  ppstaten      state_name  pce_pct_change  real_income_2023
0       ny        new york             3.2         1243246.0
1       nj      new jersey             2.1          581816.0
2       il        illinois             4.6          761677.0
3       wi       wisconsin             1.1          342613.0
4       fl         florida             3.0         1247047.0
5       nc  north carolina             2.8          591116.0
6       ct     connecticut             5.2          260583.0
7       ny        new york             3.2         1243246.0
8       ky        kentucky             1.1          230020.0
9       pa    pennsylvania            -0.1          761197.0


## Select Features

Based on SHED 2024 codebook, selecting key demographic and economic features

In [82]:
feature_columns = [
    'ppage',           # Age
    'pphhsize',        # Household size
    'ppkid017',        # Number of children under 18
    'ppeducat',        # Education level (4 categories)
    'ppethm',          # Race/ethnicity
    'ppgender',        # Gender
    'ppinc7',          # Household income (7 categories)
    
    'EF1',             # 3-month emergency fund (Yes/No)
    
    'pce_pct_change',
    'income_pct_change',
    'real_income_2023',
]

available_features = [col for col in feature_columns if col in df.columns]
print(f"Available features: {len(available_features)} out of {len(feature_columns)}")
print(f"Missing features: {set(feature_columns) - set(available_features)}")

X = df[available_features].copy()
y = df['can_cover_400'].copy()

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")

Available features: 11 out of 11
Missing features: set()

Feature matrix shape: (12295, 11)
Target shape: (12295,)


In [83]:
print("Converting categorical columns in df...")

education_mapping = {
    'No high school diploma or GED': 1,
    'High school graduate (high school diploma or the equivalent GED)': 2,
    "Some college or Associate's degree": 3,
    "Bachelor's degree or higher": 4
}
df['ppeducat'] = df['ppeducat'].map(education_mapping)
print(f"Education: {df['ppeducat'].notna().sum()} values converted")

race_mapping = {
    'White, Non-Hispanic': 1,
    'Black, Non-Hispanic': 2,
    'Other, Non-Hispanic': 3,
    'Hispanic': 4,
    '2+ Races, Non-Hispanic': 5
}
df['ppethm'] = df['ppethm'].map(race_mapping)
print(f"Race: {df['ppethm'].notna().sum()} values converted")

gender_mapping = {'Male': 1, 'Female': 2}
df['ppgender'] = df['ppgender'].map(gender_mapping)
print(f"Gender: {df['ppgender'].notna().sum()} values converted")

income_mapping = {
    'Less than $10,000': 1,
    '$10,000 to $24,999': 2,
    '$25,000 to $49,999': 3,
    '$50,000 to $74,999': 4,
    '$75,000 to $99,999': 5,
    '$100,000 to $149,999': 6,
    '$150,000 or more': 7
}
df['ppinc7'] = df['ppinc7'].map(income_mapping)
print(f"Income: {df['ppinc7'].notna().sum()} values converted")

df['EF1'] = (df['EF1'] == 'Yes').astype(int)
print(f"EF1: {df['EF1'].notna().sum()} values converted")

print("\n✓ All categorical variables converted to numeric!")

feature_columns = [
    'ppage', 'pphhsize', 'ppkid017', 'ppeducat', 'ppethm', 'ppgender', 'ppinc7',
    'EF1', 'pce_pct_change', 'income_pct_change', 'real_income_2023'
]

X = df[feature_columns].copy()
y = df['can_cover_400'].copy()

print(f"\nFeature matrix: {X.shape}")
print(f"Missing values in X: {X.isnull().sum().sum()}")
print("\nMissing by column:")
print(X.isnull().sum()[X.isnull().sum() > 0])

Converting categorical columns in df...
Education: 12295 values converted
Race: 12295 values converted
Gender: 12295 values converted
Income: 12295 values converted
EF1: 12295 values converted

✓ All categorical variables converted to numeric!

Feature matrix: (12295, 11)
Missing values in X: 603

Missing by column:
pce_pct_change       201
income_pct_change    201
real_income_2023     201
dtype: int64


## Handle Missing Values

In [84]:
print("Missing values before handling:")
print(X.isnull().sum()[X.isnull().sum() > 0])

valid_indices = ~y.isnull()
X = X[valid_indices]
y = y[valid_indices]

print(f"\nRecords after removing missing target: {len(X)}")

for col in X.columns:
    if X[col].isnull().sum() > 0:
        median_val = X[col].median()
        X[col].fillna(median_val, inplace=True)
        print(f"{col}: Filled {X[col].isnull().sum()} missing with median {median_val:.2f}")

print(f"\nDataset shape after cleaning: {X.shape}")
print(f"Missing values after handling: {X.isnull().sum().sum()}")

if X.isnull().sum().sum() == 0:
    print("\n✓ All missing values handled successfully!")
else:
    print("\n✗ Still have missing values!")

Missing values before handling:
pce_pct_change       201
income_pct_change    201
real_income_2023     201
dtype: int64

Records after removing missing target: 12295
pce_pct_change: Filled 0 missing with median 2.60
income_pct_change: Filled 0 missing with median 1.90
real_income_2023: Filled 0 missing with median 581816.00

Dataset shape after cleaning: (12295, 11)
Missing values after handling: 0

✓ All missing values handled successfully!


## Train/Test Split

In [85]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE,
    stratify=y
)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")
print(f"\nClass distribution in train:")
print(y_train.value_counts(normalize=True))
print(f"\nClass distribution in test:")
print(y_test.value_counts(normalize=True))

Training set: 9,836 samples
Test set: 2,459 samples

Class distribution in train:
can_cover_400
0    0.56344
1    0.43656
Name: proportion, dtype: float64

Class distribution in test:
can_cover_400
0    0.563237
1    0.436763
Name: proportion, dtype: float64


## Feature Scaling

In [86]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Features scaled using StandardScaler")
print(f"Training set shape: {X_train_scaled.shape}")
print(f"Test set shape: {X_test_scaled.shape}")

Features scaled using StandardScaler
Training set shape: (9836, 11)
Test set shape: (2459, 11)


## Save Processed Data

In [87]:
import pickle

np.save('../data/processed/X_train_scaled.npy', X_train_scaled)
np.save('../data/processed/X_test_scaled.npy', X_test_scaled)
np.save('../data/processed/y_train.npy', y_train)
np.save('../data/processed/y_test.npy', y_test)

with open('../data/processed/feature_names.pkl', 'wb') as f:
    pickle.dump(X.columns.tolist(), f)
with open('../data/processed/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Processed data saved to ../data/processed/")
print("\nPreprocessing Summary:")
print(f"- Features: {X_train_scaled.shape[1]}")
print(f"- Training samples: {X_train_scaled.shape[0]:,}")
print(f"- Test samples: {X_test_scaled.shape[0]:,}")
print(f"- Target: can_cover_400 (binary)")
print(f"- Can cover $400: {y.mean()*100:.1f}%")
print(f"- Cannot cover $400: {(1-y.mean())*100:.1f}%")

Processed data saved to ../data/processed/

Preprocessing Summary:
- Features: 11
- Training samples: 9,836
- Test samples: 2,459
- Target: can_cover_400 (binary)
- Can cover $400: 43.7%
- Cannot cover $400: 56.3%
